### From articles "Options for smart investor"

https://dzen.ru/id/5f6b2336d03ed90da6bf9d00


In [1]:
# Prepare data and option_lib to use in Google Collab
!bash -c '\
OPT_LIB_PATH="/content/option_lib"; \
if [[ ! (-z "${COLAB_JUPYTER_IP}" || -d "${OPT_LIB_PATH}" ) ]]; then \
  git clone https://github.com/akumidv/option_lib.git "${OPT_LIB_PATH}"; \
  DATA_PATH="/content/data"; \
  mkdir -p "${DATA_PATH}/DERIBIT/BTC-USD/EOD/futures"; \
  mkdir -p "${DATA_PATH}/DERIBIT/BTC-USD/EOD/options"; \
  gdown --fuzzy "https://drive.google.com/file/d/1n2T4jBHeanGLtBLlLWgYIHzK9hYsGT97/view?usp=sharing"  -O "/content/data/DERIBIT/BTC-USD/EOD/futures/2025.parquet" ;\
  gdown --fuzzy "https://drive.google.com/file/d/10Eo_4oNSlx0rbe2efn5OXToONAi5nPhV/view?usp=sharing"  -O "/content/data/DERIBIT/BTC-USD/EOD/options/2025.parquet"  ;\
  export VENV_PATH="${OPT_LIB_PATH}/.venv";\
  pip install poetry ;\
  cd ${OPT_LIB_PATH} ;\
  POETRY_VIRTUALENVS_IN_PROJECT=true poetry install --no-interaction;\
fi'
import sys, os

if os.path.isdir('/content/option_lib') and '/content/option_lib/src' not in sys.path:
    sys.path.extend(
        ['/content/option_lib/src', '/content/option_lib/.venv/'])  # Not work yet - poetry install somewhere else
    os.environ['DATA_PATH'] = '/content/data'


In [2]:
import datetime
import pandas as pd
from chart_studio import plotly
from plotly import graph_objs as go
from plotly.offline import iplot, init_notebook_mode
#import plotly.express as px

from options_assembler.provider import PandasLocalFileProvider, RequestParameters
from options_assembler.entities import OptionType, OptionPriceStatus
from options_assembler import Option
from options_assembler.entities import LegType, OptionLeg

init_notebook_mode(connected=True)

pd.set_option("display.max_rows", 20, "display.max_columns", 30)

In [3]:
deribit_provider = PandasLocalFileProvider('DERIBIT', os.environ.get('DATA_PATH', '../../data'))
cur_dt = datetime.date.today()
provider_params = RequestParameters(period_to=cur_dt.year)
btc = Option(deribit_provider, 'BTC', provider_params,
             option_columns=['timestamp', 'expiration_date', 'strike', 'option_type', 'price',
                             'underlying_expiration_date', 'exchange_price', 'exchange_iv',
                             'underlying_price'])  #, 'iv', 'delta', 'gamma', 'vega', 'theta', 'quick_delta', 'contract_size'])


In [4]:
btc.df_hist

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price
0,2025-03-07 00:00:00+00:00,2025-03-10 00:00:00+00:00,100000.0,c,26.023704,2025-03-10 00:00:00+00:00,19.236722,76.47,86725.488807
1,2025-03-08 00:00:00+00:00,2025-03-10 00:00:00+00:00,100000.0,c,8.615089,2025-03-10 00:00:00+00:00,1.406844,80.67,86146.710485
2,2025-03-09 00:00:00+00:00,2025-03-10 00:00:00+00:00,100000.0,c,8.058437,2025-03-10 00:00:00+00:00,0.000000,110.98,80579.920875
3,2025-03-10 00:00:00+00:00,2025-03-10 00:00:00+00:00,100000.0,c,8.223679,2025-03-10 00:00:00+00:00,0.000000,110.98,82236.790000
5,2025-03-07 00:00:00+00:00,2025-03-10 00:00:00+00:00,100000.0,p,14445.122258,2025-03-10 00:00:00+00:00,13285.977891,76.47,86730.054757
...,...,...,...,...,...,...,...,...,...
35223,2025-03-09 00:00:00+00:00,2025-03-09 00:00:00+00:00,98000.0,c,8.597618,2025-03-09 00:00:00+00:00,0.000000,87.16,85976.180000
35225,2025-03-06 00:00:00+00:00,2025-03-09 00:00:00+00:00,98000.0,p,8544.061550,2025-03-09 00:00:00+00:00,8509.239553,94.12,89937.750000
35226,2025-03-07 00:00:00+00:00,2025-03-09 00:00:00+00:00,98000.0,p,12427.527172,2025-03-09 00:00:00+00:00,11278.250862,87.16,86734.688447
35227,2025-03-08 00:00:00+00:00,2025-03-09 00:00:00+00:00,98000.0,p,11974.973710,2025-03-09 00:00:00+00:00,11850.689852,87.16,86148.732516


In [5]:
df_fut = btc.df_fut
df_fut.head(2)

,timestamp,expiration_date,price
0,2025-02-11 00:00:00+00:00,2025-02-14 00:00:00+00:00,95800.0
1,2025-02-12 00:00:00+00:00,2025-02-14 00:00:00+00:00,97942.5


In [6]:
btc.enrichment.add_column('underlying_price').add_column('time_value')
btc.df_hist.head(2)

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value
0,2025-03-07 00:00:00+00:00,2025-03-10 00:00:00+00:00,100000.0,c,26.023704,2025-03-10 00:00:00+00:00,19.236722,76.47,86725.488807,0.0,26.023704
1,2025-03-08 00:00:00+00:00,2025-03-10 00:00:00+00:00,100000.0,c,8.615089,2025-03-10 00:00:00+00:00,1.406844,80.67,86146.710485,0.0,8.615089


In [7]:
btc.enrichment.add_column('price_status')
btc.df_hist[btc.df_hist['price_status'] == OptionPriceStatus.ATM.code].head(2)

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value,price_status
80,2025-03-09 00:00:00+00:00,2025-03-10 00:00:00+00:00,80500.0,c,846.135885,2025-03-10 00:00:00+00:00,891.962605,87.59,80579.920875,79.920875,766.21501,atm
83,2025-03-09 00:00:00+00:00,2025-03-10 00:00:00+00:00,80500.0,p,725.259330,2025-03-10 00:00:00+00:00,812.037415,87.59,80579.920875,0.000000,725.25933,atm


In [8]:
df_opt = btc.df_hist
df_opt

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value,price_status
0,2025-03-07 00:00:00+00:00,2025-03-10 00:00:00+00:00,100000.0,c,26.023704,2025-03-10 00:00:00+00:00,19.236722,76.47,86725.488807,0.000000,26.023704,otm
1,2025-03-08 00:00:00+00:00,2025-03-10 00:00:00+00:00,100000.0,c,8.615089,2025-03-10 00:00:00+00:00,1.406844,80.67,86146.710485,0.000000,8.615089,otm
2,2025-03-09 00:00:00+00:00,2025-03-10 00:00:00+00:00,100000.0,c,8.058437,2025-03-10 00:00:00+00:00,0.000000,110.98,80579.920875,0.000000,8.058437,otm
3,2025-03-10 00:00:00+00:00,2025-03-10 00:00:00+00:00,100000.0,c,8.223679,2025-03-10 00:00:00+00:00,0.000000,110.98,82236.790000,0.000000,8.223679,otm
5,2025-03-07 00:00:00+00:00,2025-03-10 00:00:00+00:00,100000.0,p,14445.122258,2025-03-10 00:00:00+00:00,13285.977891,76.47,86730.054757,13269.945243,1175.177015,itm
...,...,...,...,...,...,...,...,...,...,...,...,...
35223,2025-03-09 00:00:00+00:00,2025-03-09 00:00:00+00:00,98000.0,c,8.597618,2025-03-09 00:00:00+00:00,0.000000,87.16,85976.180000,0.000000,8.597618,otm
35225,2025-03-06 00:00:00+00:00,2025-03-09 00:00:00+00:00,98000.0,p,8544.061550,2025-03-09 00:00:00+00:00,8509.239553,94.12,89937.750000,8062.250000,481.811550,itm
35226,2025-03-07 00:00:00+00:00,2025-03-09 00:00:00+00:00,98000.0,p,12427.527172,2025-03-09 00:00:00+00:00,11278.250862,87.16,86734.688447,11265.311553,1162.215619,itm
35227,2025-03-08 00:00:00+00:00,2025-03-09 00:00:00+00:00,98000.0,p,11974.973710,2025-03-09 00:00:00+00:00,11850.689852,87.16,86148.732516,11851.267484,123.706226,itm


In [9]:
# search for longest timeseries expiration_date to demonstrate time price descreasing
settlement_date = df_opt['timestamp'].max()
expiration_date = btc.chain.get_settlement_longest_period_expired_date(settlement_date)
settlement_date, expiration_date

(Timestamp('2025-03-22 00:00:00+0000', tz='UTC'),
 Timestamp('2025-03-28 00:00:00+0000', tz='UTC'))

In [10]:
df_opt_chain = btc.chain.select_chain(settlement_date, expiration_date)

df_opt_chain

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value,price_status
25505,2025-03-22 00:00:00+00:00,2025-03-28 00:00:00+00:00,100000.0,c,16.824268,2025-03-28 00:00:00+00:00,21.053048,56.21,84172.60,0.00,16.824268,otm
25545,2025-03-22 00:00:00+00:00,2025-03-28 00:00:00+00:00,100000.0,p,16656.025320,2025-03-28 00:00:00+00:00,15839.419094,56.21,84172.14,15827.86,828.165320,itm
25585,2025-03-22 00:00:00+00:00,2025-03-28 00:00:00+00:00,102000.0,c,16.824268,2025-03-28 00:00:00+00:00,20.484388,61.55,84172.14,0.00,16.824268,otm
25625,2025-03-22 00:00:00+00:00,2025-03-28 00:00:00+00:00,102000.0,p,17875.784750,2025-03-28 00:00:00+00:00,17837.644976,61.55,84172.14,17827.86,47.924750,itm
25627,2025-03-22 00:00:00+00:00,2025-03-28 00:00:00+00:00,103000.0,c,25.236402,2025-03-28 00:00:00+00:00,15.019024,61.95,84172.60,0.00,25.236402,otm
...,...,...,...,...,...,...,...,...,...,...,...,...
29153,2025-03-22 00:00:00+00:00,2025-03-28 00:00:00+00:00,95000.0,p,11104.016880,2025-03-28 00:00:00+00:00,10896.269978,50.36,84172.60,10827.40,276.616880,itm
29193,2025-03-22 00:00:00+00:00,2025-03-28 00:00:00+00:00,96000.0,c,67.297072,2025-03-28 00:00:00+00:00,58.618273,51.73,84172.60,0.00,67.297072,otm
29233,2025-03-22 00:00:00+00:00,2025-03-28 00:00:00+00:00,96000.0,p,13585.596410,2025-03-28 00:00:00+00:00,11879.396919,51.73,84172.14,11827.86,1757.736410,itm
29273,2025-03-22 00:00:00+00:00,2025-03-28 00:00:00+00:00,98000.0,c,50.472804,2025-03-28 00:00:00+00:00,33.730975,53.77,84172.60,0.00,50.472804,otm


Futures types:
* Currency
* Index
* Commodities
* Stock
* Percent

Is crypto new type or currency?


BRN specification https://www.cmegroup.com/markets/energy/crude-oil/brent-crude-oil.html

Option chain https://www.barchart.com/futures/quotes/CB*0/options?futuresOptionsView=merged

In [11]:
atm_nearest_strikes = btc.chain.get_atm_nearest_strikes()
atm_strike = atm_nearest_strikes[0]
atm_strike

np.float64(84000.0)

In [12]:
atm_strike_df = df_opt_chain[df_opt_chain['strike'] == atm_strike]
atm_strike_row = atm_strike_df[atm_strike_df['option_type'] == OptionType.CALL.code].iloc[0]
atm_strike_df

,timestamp,expiration_date,strike,option_type,price,underlying_expiration_date,exchange_price,exchange_iv,underlying_price,intrinsic_value,time_value,price_status
28569,2025-03-22 00:00:00+00:00,2025-03-28 00:00:00+00:00,84000.0,c,1892.73015,2025-03-28 00:00:00+00:00,1877.870115,40.99,84172.6,172.6,1720.13015,atm
28595,2025-03-22 00:00:00+00:00,2025-03-28 00:00:00+00:00,84000.0,p,1724.48747,2025-03-28 00:00:00+00:00,1705.884036,40.99,84172.6,0.0,1724.48747,atm


In [13]:
df_desk = btc.chain.get_desk(
    option_columns=['price', 'exchange_price', 'exchange_iv', 'price_status',
                    'intrinsic_value', 'time_value', 'timestamp', 'expiration_date', 'strike'])
res_col = ['time_value_call', 'intrinsic_value_call', 'exchange_iv_call', 'exchange_price_call', 'price_call',
           'price_status_call',
           'strike',
           'price_status_put', 'price_put', 'exchange_price_put', 'exchange_iv_put', 'intrinsic_value_put',
           'time_value_put',
           'timestamp', 'underlying_price', 'expiration_date', 'underlying_expiration_date']
df_desk[df_desk['strike'].isin(atm_nearest_strikes[:10])][res_col]

,time_value_call,intrinsic_value_call,exchange_iv_call,exchange_price_call,price_call,price_status_call,strike,price_status_put,price_put,exchange_price_put,exchange_iv_put,intrinsic_value_put,time_value_put,timestamp,underlying_price,expiration_date,underlying_expiration_date
24,-872.495580,6172.14,47.20,6426.474164,5299.644420,itm,78000.0,otm,243.951886,258.105301,47.20,0.00,243.951886,2025-03-22 00:00:00+00:00,84172.60,2025-03-28 00:00:00+00:00,2025-03-28 00:00:00+00:00
25,-872.495580,6172.14,47.20,6426.474164,5299.644420,itm,78000.0,otm,243.951886,258.105301,47.20,0.00,243.951886,2025-03-22 00:00:00+00:00,84172.14,2025-03-28 00:00:00+00:00,2025-03-28 00:00:00+00:00
26,496.134370,4172.60,43.92,4651.186658,4668.734370,itm,80000.0,otm,504.728040,481.612337,43.92,0.00,504.728040,2025-03-22 00:00:00+00:00,84172.60,2025-03-28 00:00:00+00:00,2025-03-28 00:00:00+00:00
27,496.134370,4172.60,43.92,4651.186658,4668.734370,itm,80000.0,otm,504.728040,481.612337,43.92,0.00,504.728040,2025-03-22 00:00:00+00:00,84172.14,2025-03-28 00:00:00+00:00,2025-03-28 00:00:00+00:00
28,898.288910,2172.14,41.98,3100.409756,3070.428910,itm,82000.0,otm,967.395410,929.629134,41.98,0.00,967.395410,2025-03-22 00:00:00+00:00,84172.60,2025-03-28 00:00:00+00:00,2025-03-28 00:00:00+00:00
29,898.288910,2172.14,41.98,3100.409756,3070.428910,itm,82000.0,otm,967.395410,929.629134,41.98,0.00,967.395410,2025-03-22 00:00:00+00:00,84172.14,2025-03-28 00:00:00+00:00,2025-03-28 00:00:00+00:00
30,1266.918860,1172.60,41.31,2439.756923,2439.518860,itm,83000.0,otm,1219.759430,1268.373994,41.31,0.00,1219.759430,2025-03-22 00:00:00+00:00,84172.60,2025-03-28 00:00:00+00:00,2025-03-28 00:00:00+00:00
31,1266.918860,1172.60,41.31,2439.756923,2439.518860,itm,83000.0,otm,1219.759430,1268.373994,41.31,0.00,1219.759430,2025-03-22 00:00:00+00:00,84172.14,2025-03-28 00:00:00+00:00,2025-03-28 00:00:00+00:00
32,1720.130150,172.60,40.99,1877.870115,1892.730150,atm,84000.0,atm,1724.487470,1705.884036,40.99,0.00,1724.487470,2025-03-22 00:00:00+00:00,84172.60,2025-03-28 00:00:00+00:00,2025-03-28 00:00:00+00:00
33,1720.130150,172.60,40.99,1877.870115,1892.730150,atm,84000.0,atm,1724.487470,1705.884036,40.99,0.00,1724.487470,2025-03-22 00:00:00+00:00,84172.14,2025-03-28 00:00:00+00:00,2025-03-28 00:00:00+00:00


The time value of the option decreases by the expiration date

ITM options have a low time value (low weight) in the total option price. In fact, the deeper an option is in the money, the lower its time value and the more it resembles a linear underlying asset (i.e., the corresponding futures).

OTM options have no intrinsic value (which is understandable – they are unprofitable to execute). The entire value of an out–of-money option is the intrinsic value. It is important to understand that if the market does not overcome the strike of this option, the final expiration option price will be zero.

ATM options have the maximum weight of the time value in the option price and the maximum non-linearity. For this reason, they are the most liquid on the market. When the underlying asset moves up or down from current levels, the liquidity of these options will decrease.

In [14]:
settlement_date, expiration_date = btc.chain.get_settlement_and_expiration_date()
settlement_date, expiration_date

(Timestamp('2025-03-22 00:00:00+0000', tz='UTC'),
 Timestamp('2025-03-28 00:00:00+0000', tz='UTC'))

In [15]:
btc.chart.init(title=f'Time Value for {expiration_date.date().isoformat()} expiration')
btc.chart.price.time_values(expiration_date=expiration_date, name='ATM')
btc.chart.show()

In [16]:
# add values to chart
btc.chart.init(title=f'Time Value for {expiration_date.date().isoformat()} expiration')
btc.chart.price.time_values_for_strike(expiration_date=expiration_date, strike=atm_nearest_strikes[5],
                                       name=f'strike {atm_nearest_strikes[3]}')
btc.chart.price.time_values_for_distance(expiration_date=expiration_date, distance=-5000, name='ATM distance -$5000')
btc.chart.show()

In [17]:
df_time_value_strike = btc.analytic.price.time_value_series_by_strike_to_atm_distance(strike=atm_nearest_strikes[5])
df_time_value_strike.iloc[[0, int(len(df_time_value_strike) / 2), -1]]

,timestamp,strike,time_value
5,2025-03-19 00:00:00+00:00,91000.0,408.367255
48,2025-03-21 00:00:00+00:00,88000.0,8.405510
75,2025-03-22 00:00:00+00:00,88000.0,8.412134


In [18]:
df_time_value_atm = btc.analytic.price.time_value_series_by_atm_distance(distance=0)
df_time_value_atm

,timestamp,strike,time_value
9,2025-03-19 00:00:00+00:00,87000.0,1477.073050
26,2025-03-20 00:00:00+00:00,84000.0,699.810957
62,2025-03-21 00:00:00+00:00,84000.0,327.775439
80,2025-03-22 00:00:00+00:00,84000.0,155.211023


In [19]:
auto_expiration_date = btc.chain.df_chain['expiration_date'].min()
btc.chart.init(title=f'Time Value from prepared dataframes for {auto_expiration_date.date().isoformat()} expiration')
btc.chart.price.time_values([df_time_value_strike, df_time_value_atm], [f'Strike cur ATM {atm_strike}', 'ATM'],
                            expiration_date=expiration_date)
btc.chart.show()

## Non-linearity of options

When buying an option, we have the so–called non-linearity of profit - in a positive scenario (growth of the underlying asset) we earn more, in a negative scenario (a decline in the underlying asset), we lose less with the same amount of futures change.

In [20]:
strikes = btc.chain.get_atm_nearest_strikes()
max_strike_risk_2 = list(filter(lambda x: atm_strike < x < atm_strike * 1.05, strikes))[-1]
strike_start = sorted(filter(lambda x: x <= min(atm_strike, max_strike_risk_2) * 0.90, strikes))[-1]
strike_end = sorted(filter(lambda x: x >= max(atm_strike, max_strike_risk_2) * 1.1, strikes))[0]
strike_start, strike_end

(np.float64(75000.0), np.float64(98000.0))

In [21]:
fut_legs = [OptionLeg(strike=0, lots=1, type=LegType.FUTURE)]
fut_risk_profile, fut_risk_legs_pnl = btc.analytic.risk.chain_payoff(fut_legs)
fut_risk_profile.head(2)

,strike,risk_pnl
0,20000.0,-64172.60
1,30000.0,-54172.14


In [22]:
call_legs = [OptionLeg(strike=atm_strike, lots=1, type=LegType.OPTION_CALL)]
call_risk_profile, call_risk_legs_pnl = btc.analytic.risk.chain_payoff(call_legs)
call_legs_1 = [OptionLeg(strike=max_strike_risk_2, lots=1, type=LegType.OPTION_CALL)]
call_risk_profile_1, df_legs_risk_profile = btc.analytic.risk.chain_payoff(call_legs_1)

In [24]:
fut_view = fut_risk_profile[
    (fut_risk_profile['strike'] > strike_start) & (fut_risk_profile['strike'] < strike_end)]
call_view_1 = call_risk_profile[
    (call_risk_profile['strike'] > strike_start) & (call_risk_profile['strike'] < strike_end)]
call_view_2 = call_risk_profile_1[
    (call_risk_profile_1['strike'] > strike_start) & (call_risk_profile_1['strike'] < strike_end)]
data = [
    go.Scatter(x=fut_view['strike'], y=fut_view['risk_pnl'].to_list(), mode='lines', name='fut'),

    go.Scatter(x=call_view_1['strike'], y=call_view_1['risk_pnl'].to_list(), mode='lines',
               name=f'call atm {call_legs[0].strike}'),
    go.Scatter(x=call_view_2['strike'], y=call_view_2['risk_pnl'], mode='lines',
               name=f'call {call_legs_1[0].strike}'),
    # go.Scatter(x=call_view_2['strike'], y=call_view_2['risk_pnl_premium'], mode='lines',
    #            name=f'call premium {call_legs_1[0].strike}')
]  # markers, lines
iplot(data)

In [26]:
short_long_legs = [OptionLeg(strike=atm_strike, lots=1, type=LegType.OPTION_PUT)]
put_atm_risk_profile, put_atm_risk_legs_pnl = btc.analytic.risk.chain_payoff(short_long_legs)
put_atm_risk_profile

,strike,risk_pnl
0,20000.0,62275.51253
1,30000.0,52275.51253
2,40000.0,42275.51253
3,45000.0,37275.51253
4,50000.0,32275.51253
...,...,...
47,180000.0,-1724.48747
48,200000.0,-1724.48747
49,220000.0,-1724.48747
50,250000.0,-1724.48747


Non-linearity of profit. We make more profit at the turning point than at futures по премии.

In [28]:
fut_view = fut_risk_profile[
    (fut_risk_profile['strike'] > strike_start) & (fut_risk_profile['strike'] < strike_end)]
put_view_1 = put_atm_risk_profile[
    (put_atm_risk_profile['strike'] > strike_start) & (put_atm_risk_profile['strike'] < strike_end)]
data = [
    go.Scatter(x=fut_view['strike'], y=fut_view['risk_pnl'].to_list(), mode='lines', name='fut'),

    go.Scatter(x=put_view_1['strike'], y=put_view_1['risk_pnl'].to_list(), mode='lines',
               name=f'call atm {short_long_legs[0].strike}'),
    # go.Scatter(x=put_view_1['strike'], y=put_view_1['risk_pnl_premium'], mode='lines',
    #            name=f'call atm premium {short_long_legs[0].strike}'),
]  # markers, lines
iplot(data)
# TODO move to chart above calc

**THE RISKS OF BUYING OPTIONS**

Risks of Long Call:
- Decreasing the price of the underlying asset (UA). The position earns on the growth of the underlying asset
- The expiration date is approaching. The fewer days before the expiration of an option, the cheaper its cost while maintaining the same underlying asset price and volatility values.
- Reducing market volatility. With increasing volatility in the market, the probability of moving position in the money increases and, accordingly, options become more expensive (with other parameters constant). With a decrease in volatility, option premiums are getting cheaper. Thus, it is not profitable for option buyers to reduce volatility.

Risks of Long Put:
- Increasing the price of the underlying asset. The position earns on the decreasing the underlying asset price
- The expiration date is approaching.
- Reducing market volatility.

The asymmetric dynamics of the stock market. Usually, market growth occurs more smoothly and relatively slowly, and the decline is usually very sharp. Due to the fact that the decline is usually faster than the growth, the purchase of a Put option can be more effective than the purchase of a Call.

For both strategies, reducing market volatility works against the position, as the Call and Put premiums in the market become cheaper in this case.
When the stock market declines, volatility tends to increase (which is understandable, the decline occurs more quickly), and when it increases, it usually decreases.

By buying a Call option, we earn money on the growth of the UA, on the decrease in volatility that usually accompanies the growth of the instrument, which works against our position.

By buying a Put option, we have the opposite picture. We make money from a declining market, and the resulting general increase in market volatility helps us make even more profit.

**Thus, buying a Put option is a more effective strategy than buying a Call option, both because of the faster decline in the market and due to the impact of market volatility on our position.**

When buying a longer-term option, we have less risk, but this position will be less profitable if the market moves in our direction.

When buying a shorter-term option, we have a greater negative time impact on the position, but in the case of a trend towards us, the option will rise in price faster.

## Profit ratio
TODO to chart

In [26]:
df_chain_call = df_opt_chain[df_opt_chain['option_type'] == 'c'].sort_values('strike').reset_index(drop=True)
current_fut_price = df_chain_call.iloc[0]['underlying_price']
target_price = 110_000
call_atm = df_chain_call[df_chain_call['price_status'] == 'atm'].iloc[0]
call_itm = df_chain_call[df_chain_call['strike'] < call_atm['strike'] - (target_price - call_atm['strike'])].iloc[-1]
call_otm = df_chain_call[df_chain_call['strike'] > (target_price - call_atm['strike'])/2 + call_atm['strike']].iloc[0]

{'fut': current_fut_price,  'target': target_price, 'atm': call_atm['strike'], 'itm': call_itm['strike'], 'otm': call_otm['strike']}

{'fut': np.float64(87729.7),
 'target': 110000,
 'atm': np.float64(88000.0),
 'itm': np.float64(65000.0),
 'otm': np.float64(100000.0)}

In [27]:
df_profit = pd.DataFrame.from_dict({
    'future':                       {'Price': current_fut_price, 'Expiration price': target_price},
    f'ITM {call_itm["strike"]}':    {'Price': call_itm['price'], 'Expiration price': target_price - call_itm['strike']},
    f'ATM {call_atm["strike"]}':    {'Price': call_atm['price'], 'Expiration price': target_price - call_atm['strike']},
    f'OTM {call_otm["strike"]}':    {'Price': call_otm['price'], 'Expiration price': target_price - call_otm['strike']}
})

df_profit.index.name = 'Metric'
df_profit.reset_index(drop=False, inplace=True)
columns = [col for col in df_profit.columns if col != 'Metric']
profit_diff = df_profit[columns].reset_index(drop=True).diff().dropna()

profit_diff['Metric'] = 'Profit'

df_profit = pd.concat([df_profit, profit_diff], ignore_index=True)
profit_ratio = df_profit[columns].iloc[2]/df_profit[columns].iloc[0]
profit_ratio['Metric'] = 'Profit ratio'
df_profit = pd.concat([df_profit, profit_ratio.to_frame().T], ignore_index=True)
df_profit

,Metric,future,ITM 65000.0,ATM 88000.0,OTM 100000.0
0,Price,87729.7,20681.709785,4853.424495,1399.18544
1,Expiration price,110000.0,45000.0,22000.0,10000.0
2,Profit,22270.3,24318.290215,17146.575505,8600.81456
3,Profit ratio,0.253851,1.175836,3.532882,6.147015


**Recommendation**:

If there are waiting underlying asser grow, it has sense to buy call option with strike in the middle of awaiting price

If time has a negative effect on our position, it means that we should be in the position for a minimum of time!

He stays in the position of purchased options for as little time as possible. We entered at the beginning of the movement:

1. either you made a profit quickly and exited the position (or you rebuilt it in the hope of making even more profit with less risk)

2. either they made a mistake with the direction of movement and exited the position with a loss (or they rebuilt it in the hope of recapturing the loss due to the new option scheme)

The main mistake most beginners make is holding positions based on buying Call or Put options for a long time.

Set a time frame for holding a position in advance and follow it clearly.

**When should you use naked option buying in your trading?** Only if there are strong trends up or down in the shortest possible time.

## Selling options

Covered– we have sold an option and have a long position in the underlying asset in case of a Call sale or a short position in the underlying asset in case of a Put sale. The position in the underlying asset covers your risks on the option sold.

From a mathematical point of view, the probability of making a profit when selling options is higher than when buying options.

When selling an option, we make a profit in the amount of a premium, not only when:
- the underlying asset (UA) price changes in our direction (in this case, with a decrease in futures);
- while maintaining the UA price at the current level (sideways market movement);
- when selling an OTM option when futures move against our position (sale strike + premium earned).

We only lose money when there is a strong trend against us.


In [28]:
short_long_legs = [OptionLeg(strike=atm_strike, lots=-1, type=LegType.OPTION_CALL)]
short_long_risk_profile, short_long_risk_legs_pnl = btc.analytic.risk.chain_risk_profile(short_long_legs)
short_long_legs_otm = [OptionLeg(strike=max_strike_risk_2, lots=-1, type=LegType.OPTION_CALL)]
short_long_risk_profile_otm, short_long_risk_legs_pnl_otm = btc.analytic.risk.chain_risk_profile(short_long_legs_otm)


In [29]:
short_call_view_1 = short_long_risk_profile[
    (short_long_risk_profile['strike'] > strike_start) & (short_long_risk_profile['strike'] < strike_end)]
short_call_view_otm = short_long_risk_profile_otm[
    (short_long_risk_profile_otm['strike'] > strike_start) & (short_long_risk_profile_otm['strike'] < strike_end)]
data = [
    go.Scatter(x=fut_view['strike'], y=fut_view['risk_pnl'].to_list(), mode='lines', name='fut'),

    go.Scatter(x=short_call_view_1['strike'], y=short_call_view_1['risk_pnl'].to_list(), mode='lines',
               name=f'short call atm {short_long_legs[0].strike}'),
    go.Scatter(x=short_call_view_otm['strike'], y=short_call_view_otm['risk_pnl'], mode='lines',
               name=f'call {short_long_legs_otm[0].strike}'),
]
iplot(data)

Selling uncovered options involves three main risks:

1. BA's move against our position (price risk)

2. Increased volatility in the market

3. Possible increase in the guarantee provision for the position

Note that, unlike buying options, time works for us, not against us.


### Non-linearity of loss

When selling an option, losses increase more than profits with the same price change.

It is quite possible to get a loss when selling options not only because of the strong movement of the BA against our position, but also as a result of increased market volatility due to the increased margin requirement



In [30]:
data = [
    go.Scatter(x=fut_view['strike'], y=fut_view['risk_pnl'].to_list(), mode='lines', name='fut'),

    go.Scatter(x=short_call_view_1['strike'], y=short_call_view_1['risk_pnl'].to_list(), mode='lines',
               name=f'short call atm {short_long_legs[0].strike}'),
    go.Scatter(x=short_call_view_1['strike'], y=short_call_view_1['risk_pnl_premium'], mode='lines',
               name=f'short call atm premium {short_long_legs[0].strike}'),
]
iplot(data)
# TODO wrong premium caclulations, сделать сразу график профиля

In [31]:
data = [
    go.Scatter(x=short_long_risk_profile['strike'], y=short_long_risk_profile['risk_pnl'].to_list(), mode='lines',
               name=f'short call atm {short_long_legs[0].strike}'),
    go.Scatter(x=short_long_risk_profile['strike'], y=short_long_risk_profile['risk_pnl_premium'], mode='lines',
               name=f'short call atm premium {short_long_legs[0].strike}'),
]
iplot(data)

*Resume selling options:*

It is not profitable for us to move the BA against our position and increase market volatility in the market (which may lead, among other things, to a serious increase in warranty requirements for our options).

Time always works in favor of our position.

The risks of selling a Call option (Short Call):
* an increase in the futures price (the position earns a decrease as well as while maintaining the price at the current level)
* an increase in market volatility. As volatility increases, option premiums in the market become more expensive (while maintaining the same value of the underlying asset and the reserve before expiration)
* an increase in margin requirement. The margin requirement may increase both due to the movement of the price of the underlying asset and under the influence of market volatility.

Risks of selling a Put option (Short Put):
* lower futures prices (the position earns on growth, as well as while maintaining the price at the current level)
* increased volatility in the market. With an increase in premium volatility, options on the market become more expensive (while maintaining the same value of the underlying asset and terminating before expiration)
* an increase in the margin requirement. The margin requirement may increase both due to the movement of the price of the underlying asset and under the influence of market volatility.


By selling the Put option, we lose money on a decrease in the BA, and the resulting general increase in market volatility in most cases gives us an even greater loss on the position.

By selling a Call option, we lose money on the growth of the BA, on the reduction of volatility, which usually (I emphasize, not always) accompanies the growth of the instrument works for us and improves our financial result for the position.
